# Imports

In [ ]:
!pip install simpletransformers -q

     |████████████████████████████████| 204kB 16.8MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 1.1MB 36.9MB/s 
     |████████████████████████████████| 317kB 48.5MB/s 
     |████████████████████████████████| 71kB 10.2MB/s 
     |████████████████████████████████| 1.8MB 46.8MB/s 
     |████████████████████████████████| 7.4MB 52.1MB/s 
     |████████████████████████████████| 1.4MB 47.0MB/s 
     |████████████████████████████████| 2.9MB 52.1MB/s 
     |████████████████████████████████| 102kB 14.5MB/s 
     |████████████████████████████████| 133kB 56.7MB/s 
     |████████████████████████████████| 102kB 14.6MB/s 
     |████████████████████████████████| 163kB 56.8MB/s 
     |████████████████████████████████| 4.5MB 49.7MB/s 
     |████████████████████████████████| 112kB 58.6MB/s 
     |████████████████████████████████| 890kB 44.8MB/s 
     |████████████████████████████████| 71kB 11.1MB/s 
     |████████████████████████████████| 122kB 63.0MB

In [ ]:
import pandas as pd
import numpy as np
import torch 
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import wandb

# Load data

In [ ]:
from google.colab import drive
import glob\

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# CHANGE TO YOUR PATH
colab_resources_path = "/content/gdrive/MyDrive/projectml"

In [ ]:
data_files = glob.glob(colab_resources_path + "/*.csv")
data_files += glob.glob(colab_resources_path + "/*.py")
for data_file in data_files:
  print('Copying file {} to colab root.'.format(data_file))
  !cp "$data_file" .

Copying file /content/gdrive/MyDrive/projectml/random.csv to colab root.
Copying file /content/gdrive/MyDrive/projectml/nam.csv to colab root.
Copying file /content/gdrive/MyDrive/projectml/am.csv to colab root.
Copying file /content/gdrive/MyDrive/projectml/am_additional.csv to colab root.
Copying file /content/gdrive/MyDrive/projectml/missclassified_roberta_mcc.csv to colab root.
Copying file /content/gdrive/MyDrive/projectml/miss_5models.csv to colab root.
Copying file /content/gdrive/MyDrive/projectml/missclassified_lr_robertas_ens_mcc.csv to colab root.
Copying file /content/gdrive/MyDrive/projectml/data_preprocess.py to colab root.


Get four different datasets from train data:


*   `train_data_title` - dataframe with article titles
*   `train_data_body` - dataframe will article bodies
*   `train_data_titleplus` - dataframe with article title + first sentence
*   `train_data_all` - dataframe with article title + body



In [ ]:
from data_preprocess import getTrainData
train_data_title = getTrainData(include_random=True, random_as_2=False, shuffle=False, no_title=False, n_sentences=0) #for roberta on titles only
train_data_body = getTrainData(include_random=True, random_as_2=False, shuffle=False, no_title=True, n_sentences=-1) #for roberta on articles only
train_data_titleplus = getTrainData(include_random=True, random_as_2=False, shuffle=False, no_title=False, n_sentences=1) #for roberta on title+ 1 paraghaph only
train_data_all = getTrainData(include_random=True, random_as_2=False, shuffle=False, no_title=False, n_sentences=-1) #for roberta on all

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.



# Cross-validation for RoBERTa trained on different inputs

Specify classification model arguments 

In [ ]:
model_args = ClassificationArgs(sliding_window=True)
model_args.num_train_epochs=4
model_args.save_best_model= True
model_args.tie_value = 1
model_args.batch_size = 16
model_args.learning_rate = 2e-5
model_args.overwrite_output_dir = True
model_args.max_seq_length = 512
model_args.no_cache=True
model_args.max_grad_norm = 1
model_args.use_multiprocessing = True
model_args.manual_seed = 4
model_args.reprocess_input_data = True
model_args.evaluate_during_training = True
model_args.labels_list = [0, 1]

Function for memory cleanup, useful if several models are trained in one notebook.

In [ ]:
import gc
def memory_cleanup():
    for obj in gc.get_objects():
        if torch.is_tensor(obj):
            del obj
    gc.collect()
    torch.cuda.empty_cache()
    !rm -r outputs

6-fold cross-validation, four models are trained at the same time. 
RoBERTa-base is trained on:

*   full data
*   title only
*   text only
*   title + first sentence



In [ ]:
n=6
seed=42
kf = KFold(n_splits=n, random_state=seed, shuffle=True)
acc, prec, rec, f1, mccs, wrongs = [[] for i in range(4)], [[] for i in range(4)],[[] for i in range(4)],[[] for i in range(4)],[[] for i in range(4)],[[] for i in range(4)]
miss_rb = [np.array([]) for i in range(4)]
predictions_rb=[[] for i in range(4)]
for train_index, val_index in kf.split(train_data_title): 
    memory_cleanup()

    train_df_t = train_data_title.iloc[train_index]
    val_df_t = train_data_title.iloc[val_index]
    train_df_b = train_data_body.iloc[train_index]
    val_df_b = train_data_body.iloc[val_index]
    train_df_tp = train_data_titleplus.iloc[train_index]
    val_df_tp = train_data_titleplus.iloc[val_index]
    train_df_a = train_data_all.iloc[train_index]
    val_df_a = train_data_all.iloc[val_index]

    inputs = [[train_df_t,val_df_t],[train_df_b,val_df_b],[train_df_tp,val_df_tp],[train_df_a,val_df_a]]
    for num, data in enumerate(inputs):

        model = ClassificationModel('roberta', 'roberta-base', args=model_args)
        model.train_model(data[0], eval_df=data[1], acc=matthews_corrcoef)
        result, model_outputs, wrong_predictions = model.eval_model(data[1], acc=matthews_corrcoef) 

        predictions_rb[num] = np.array([np.rint(np.mean(np.argmax(j, axis=1))) for j in model_outputs]).astype(int)

        miss_rb[num] = np.concatenate((miss_rb[num], data[1].iloc[np.where(data[1].label != predictions_rb[num])].index)).astype(int)

        acc[num].append(accuracy_score(data[1].label, predictions_rb[num]))
        prec[num].append(precision_score(data[1].label, predictions_rb[num]))
        rec[num].append(recall_score(data[1].label, predictions_rb[num]))
        f1[num].append(f1_score(data[1].label, predictions_rb[num]))
        mccs[num].append(matthews_corrcoef(data[1].label, predictions_rb[num]))
        memory_cleanup()

/usr/local/lib/python3.6/dist-packages/torch/distributed/distributed_c10d.py:126: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn("torch.distributed.reduce_op is deprecated, please use "


rm: cannot remove 'outputs': No such file or directory


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:387: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


...

/usr/local/lib/python3.6/dist-packages/torch/distributed/distributed_c10d.py:126: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn("torch.distributed.reduce_op is deprecated, please use "
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceCla

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:966: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."



0.9104120495514275


/usr/local/lib/python3.6/dist-packages/torch/distributed/distributed_c10d.py:126: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn("torch.distributed.reduce_op is deprecated, please use "


Mean accuracy, precision, recall, f1-score and mcc score for each model.

In [ ]:
for num, i in enumerate(['title', 'body', 'title_plus', 'all']):
  print(i)
  print('Accuracy score: ', np.mean(acc[num]))
  print('Precision score: ', np.mean(prec[num]))
  print('Recall score: ', np.mean(rec[num]))
  print('F1 score: ', np.mean(f1[num]))
  print('mcc score: ', np.mean(mccs[num]))

title
Accuracy score:  0.9379415519931905
Precision score:  0.9403488391087785
Recall score:  0.9362083182266684
F1 score:  0.9378689886176401
mcc score:  0.8763559931828019
body
Accuracy score:  0.9504823379202724
Precision score:  0.9516662823488587
Recall score:  0.9505458480263925
F1 score:  0.950869616138594
mcc score:  0.9012286612737972
title_plus
Accuracy score:  0.941708043694141
Precision score:  0.9382560747049097
Recall score:  0.9493038441423501
F1 score:  0.9432404442198075
mcc score:  0.8842352198196591
all
Accuracy score:  0.9498463138979524
Precision score:  0.9524302682191211
Recall score:  0.9490263408411289
F1 score:  0.9504784717966793
mcc score:  0.8999924510187757


Accuracy, precision, recall, f1-score and mcc score for each fold.

In [ ]:
for num, i in enumerate(['title', 'body', 'title_plus', 'all']):
  print(i)
  print(acc[num])
  print(prec[num])
  print(rec[num])
  print(f1[num])
  print(mccs[num])

title
[0.9172932330827067, 0.9285714285714286, 0.9624060150375939, 0.924812030075188, 0.9398496240601504, 0.9547169811320755]
[0.9136690647482014, 0.9572649572649573, 0.935251798561151, 0.9166666666666666, 0.9568345323741008, 0.9624060150375939]
[0.927007299270073, 0.8888888888888888, 0.9923664122137404, 0.9307692307692308, 0.9300699300699301, 0.9481481481481482]
[0.9202898550724637, 0.9218106995884773, 0.9629629629629629, 0.9236641221374045, 0.9432624113475178, 0.955223880597015]
[0.8344563416270928, 0.8582249666453193, 0.9265383842972261, 0.8496946609585082, 0.8796904580056691, 0.9095311475629954]
body
[0.9135338345864662, 0.9736842105263158, 0.9398496240601504, 0.9586466165413534, 0.9473684210526315, 0.969811320754717]
[0.9191176470588235, 0.9612403100775194, 0.967479674796748, 0.9407407407407408, 0.951048951048951, 0.9703703703703703]
[0.9124087591240876, 0.9841269841269841, 0.9083969465648855, 0.9769230769230769, 0.951048951048951, 0.9703703703703703]
[0.9157509157509157, 0.972549

# Save misclassified data

In [ ]:
import pickle

with open('miss_rb.pkl', 'wb') as file:
  pickle.dump(miss_rb, file)

In [ ]:
from data_preprocess import getFullRowByIndex

miss_indexes = np.unique(np.concatenate(miss_rb,axis=1)).astype(int)

miss_data = getFullRowByIndex(miss_indexes)

        print(matthews_corrcoef(i[1].label, predictions_rb[num]))
for num, i in enumerate(['title', 'body', 'title_plus', 'all']):
  miss_data['miss_rb_'+i] = np.zeros(len(miss_data.index)).astype(int)

  miss_data.loc[miss_data.loc[miss_rb[num]].index, 'miss_rb_'+i] = 1

miss_data = miss_data.drop(columns=['assessor.1', 'article.1'])

try:
  miss_data.to_csv("/content/drive/My Drive/projectml/missclassified_roberta_mcc.csv")
except:
  miss_data.to_csv("/content/gdrive/MyDrive/projectml/missclassified_roberta_mcc.csv")